<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/Attack_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attack

# Libraries

In [1]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
import re
import array
import seaborn as sns
from tqdm import tqdm

from functools import reduce
from random import random, gauss
from math import modf, pi, cos, sin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from plotly.subplots import make_subplots
from scipy.signal import savgol_filter
from scipy.stats.stats import pearsonr
from scipy.stats import spearmanr

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.mixture import GaussianMixture
import os, time, math, csv, joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import os, time, math, csv, joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

sns.set_theme()

# Load Enviroment

In [2]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

Mounted at /content/gdrive


In [3]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Change the number to change the paths.
worker = workers[0]

if worker == "Ronald":
  path = "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

# Pre-procesing

In [33]:
input_path = path + "/Spectrum/CNN/new_working_df.csv"

working_df = pd.read_csv(input_path)
working_df['Distance_km'] = working_df['Distance_km'].astype(int)

# Utils

## Generate Train/Test data

In [4]:
def create_df(input_data_path, distances, nsymbols = 2048, span_length=80, min_dist = 0, max_dist = 3000):
  X=None
  Y=[]
  colnames=['i'+str(i) for i in range(nsymbols)]

  for d in distances:
    dist=d*span_length
    if dist<min_dist or dist>max_dist: continue
    filename='consts_'+str(d)+'span.csv'
    df_aux=pd.read_csv(input_data_path+'/'+filename, sep=",", header=None)
    df_aux = df_aux.T
    df_aux.columns=colnames
    Y=Y+[dist]*df_aux.shape[0]
    if X is None: X=df_aux
    else: X=X.append(df_aux)
  return X, Y

In [25]:
def create_df_v2(working_df, distances, nsymbols = 2048, span_length=80, min_dist = 0, max_dist = 3000):
  X=None
  Y=np.array([])

  for d in distances:
    distance_working_df = working_df.loc[working_df['Distance_km'] == d]
    
    d_df = distance_working_df.iloc[:, 3].to_numpy()
    data_df = distance_working_df.iloc[:, 6:distance_working_df.shape[1]]
    Y = np.append(Y, d_df)

    if X is None: X=data_df
    else: X=X.append(data_df)
  Y = Y.astype('int').tolist()

  return X, Y

In [26]:
#input_data_path=path+"/rawData"
input_path = path + "/Spectrum/CNN/new_working_df.csv"
#distances=[i for i in range(1,26)]
distances = [x*80 for x in range(1, 26)]
X, Y = create_df_v2(working_df, distances, nsymbols = 2048, span_length=80,  min_dist = 0, max_dist = 3000)

In [27]:
print(Y)

[80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240,

In [28]:
display(X)

,-32.15625,-32.125,-32.09375,-32.0625,-32.03125,-32.0,-31.96875,-31.9375,-31.90625,-31.875,...,32.125,32.15625,32.1875,32.21875,32.25,32.28125,32.3125,32.34375,32.375,32.40625
0,-14.851402,-21.638525,-23.529105,-14.969324,-17.145545,-16.342900,-16.239795,-14.120054,-15.755819,-19.067398,...,-19.957216,-19.447732,-14.576655,-15.491358,-14.654316,-17.801658,-16.387551,-13.980891,-15.122689,-17.753853
25,-21.481096,-21.205568,-20.854126,-19.991677,-18.402227,-20.622225,-22.480008,-22.454051,-18.137350,-16.637799,...,-17.541235,-19.599321,-19.266714,-16.904699,-20.704567,-25.948745,-22.760953,-17.298135,-15.316883,-16.421025
50,-21.370852,-29.437105,-32.086535,-24.223958,-26.339000,-23.537576,-23.110697,-19.919646,-21.463850,-25.208957,...,-27.549388,-28.321971,-23.543228,-23.875223,-21.912033,-23.915182,-22.934829,-21.885358,-23.084174,-25.142280
75,-33.038353,-30.568682,-28.682813,-27.311470,-24.400452,-27.886846,-31.123211,-30.989309,-25.508209,-24.484035,...,-23.585396,-26.476201,-27.922908,-26.440117,-29.307617,-34.170560,-31.693952,-24.850096,-22.806973,-24.283206
100,-27.210933,-33.206345,-34.996978,-28.626407,-31.321302,-32.917828,-35.051870,-29.202459,-26.797388,-29.518587,...,-34.046918,-33.315631,-29.619187,-29.017395,-29.751951,-33.075750,-32.457689,-29.148764,-29.722032,-32.219497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399,-18.843547,-20.530192,-17.271023,-22.028237,-12.370868,-13.657834,-13.466080,-18.225137,-20.806093,-17.436201,...,-25.432760,-28.015109,-17.282195,-13.804389,-18.012508,-30.864317,-18.640087,-15.934693,-30.279564,-29.148301
2424,-34.865359,-16.963324,-21.229292,-15.183624,-12.850795,-16.551465,-26.206360,-19.217120,-21.431911,-19.028809,...,-18.016891,-18.806090,-19.425105,-25.694280,-27.979006,-24.798476,-27.844276,-30.270388,-14.964416,-13.160239
2449,-23.651387,-24.507387,-24.689501,-21.493006,-23.701515,-22.198794,-22.898655,-20.859601,-19.596688,-26.724999,...,-24.417255,-22.809253,-25.110462,-17.981032,-18.211118,-17.393150,-23.437428,-27.544681,-19.829203,-20.874010
2474,-23.597497,-18.165576,-22.315961,-23.090757,-13.016928,-24.549710,-15.795003,-15.683730,-28.368284,-23.680887,...,-21.686269,-22.013979,-17.506792,-19.864181,-23.401668,-27.855438,-16.789990,-23.277107,-24.994674,-31.241340


In [7]:
input_data_path=path+"/rawData"

distances=[i for i in range(1,26)]

X, Y = create_df(input_data_path, distances, nsymbols = 2048, span_length=80,  min_dist = 0, max_dist = 3000)

In [8]:
print(Y)

[80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 240, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 320, 400, 400, 400, 400, 400, 400, 400, 400, 400, 400,

In [9]:
display(X)

,i0,i1,i2,i3,i4,i5,i6,i7,i8,i9,...,i2038,i2039,i2040,i2041,i2042,i2043,i2044,i2045,i2046,i2047
0,-3.10250329028221+2.8974718146089i,-3.08519822419515+2.98467451952321i,-3.16258763128303+2.97984558299648i,1.07879018281473-3.04241675538607i,2.98754253262033+2.93928002541509i,-3.03753489675786-1.14448375850226i,3.0846017582402-2.83983223649996i,1.03529270059005+0.941187406261411i,-2.97497486604545-0.968041780938377i,-3.06094215344788-3.06159685614163i,...,1.00407667015021-1.05795855117373i,2.99512658085838-0.926750667283466i,1.06411092896718+1.02555956962374i,3.07437379185889-0.843429644287376i,2.896799443885+3.10310875655109i,-2.92935048828414-3.04043355319345i,0.979544147075317+3.05647856728028i,-1.00023102535927-3.01730414192666i,-3.10702415011799+2.90781364843713i,2.96764341678099+3.06083203420016i
1,-3.10761405468766+2.88209954082628i,-3.19533036484843+2.95874362980383i,-3.13829764175686+3.00503493552857i,1.06861837615755-2.95339956113059i,2.92921214422155+2.96188227504628i,-2.91432065303941-1.10419537488136i,3.01596531358918-2.87975183044387i,0.998438005339709+0.992890825590263i,-2.95611581894786-1.05543701378473i,-3.0188708530257-3.07654225575726i,...,1.09142929797919-0.974361196653361i,2.99292189696462-1.0161587115351i,0.985156816075877+0.990883548468719i,2.95772171640577-0.935638392241526i,2.91617636179577+3.07206541691927i,-2.93455002864304-2.99302248049629i,0.895363650001881+3.00377462740049i,-0.863137448065807-2.94913856469913i,-3.027753438257+3.10582714051129i,3.01387909455478+3.17361460219782i
2,-3.10204351668572+2.99671795878579i,-3.08856407109433+2.93202343862027i,-3.04491881048755+2.88692497524318i,1.08823852361823-2.98750440367609i,2.90697736188281+2.9758528690822i,-2.98145498382088-1.09421494578974i,3.08771003588438-2.94117387758681i,0.963331576488618+0.964855740477135i,-2.9106651176393-1.1014404929423i,-3.03198279219029-2.95534753456136i,...,0.907715239688597-1.02421970454919i,3.03546187492943-1.02360256422108i,0.893086213567738+0.966805075574357i,2.97386120778332-0.964294737203794i,2.88599504091299+3.11886939737541i,-2.90221562333671-3.07045202327123i,0.950041740922162+2.99164321430862i,-0.923820303162464-3.01903477870031i,-3.11729966991965+2.9837680721312i,2.91193728337384+3.03679452628824i
3,-3.09457408196026+2.8388859123441i,-3.08494101223556+2.88388527447527i,-3.20677263827856+2.91317189447773i,1.1673903167167-2.99848089665644i,2.94658204937336+3.00802534290194i,-2.87526098987528-1.11405630308427i,3.08259417259759-2.9194453476648i,1.06273634615093+1.10984003966738i,-3.032693679052-1.02306856691278i,-3.01473599608657-2.99577983799314i,...,1.00758460229825-0.965973467978032i,3.10968647160534-0.91623839775481i,0.938950558325145+1.00989206333647i,2.94432128245389-0.931510855582151i,2.98495085825902+3.22416237151373i,-2.98008667418847-3.17718280096277i,1.0741624539175+3.000073164185i,-0.881408685061913-2.99428343114252i,-3.10153645453798+3.03656147006742i,3.00964861698049+3.09772452239695i
4,-3.13127191492135+2.9754358260431i,-3.14772852543847+2.89217924882178i,-3.21645373194406+2.97011865484281i,1.11316751649522-2.97731443054873i,2.73524358695753+3.05948601698787i,-2.97212874519421-1.0590367110878i,3.0777047357009-2.90727677556418i,0.982013389869338+0.963790372036073i,-2.97980570056718-1.09959616069286i,-2.96660065038705-3.12778624332231i,...,0.94720034633235-0.974594174915912i,2.97682481699014-0.881520090647282i,0.988875523324602+1.03869086925536i,3.00253603823219-0.849527423903983i,2.93983404932116+3.00032841285343i,-2.81780087982184-3.03100101937592i,0.810746709950622+3.07897052838216i,-0.891457084837401-3.0717974290845i,-3.06133833488459+3.01647360713552i,2.97503513411931+3.13292798539178i
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,-2.7011621230512+2.87765912547753i,-2.80211790685203+3.20223387722402i,-3.05291419508582+2.9176937027566i,1.61806961406685-2.957283648909i,3.18257037343425+2.80947966796915i,-2.7000197148827-1.38272619000599i,2.57547211968966-2.96

In [32]:
working_df

,row,PBRS_id,Channels,Distance_km,power_dBm,#span,-32.15625,-32.125,-32.09375,-32.0625,...,32.125,32.15625,32.1875,32.21875,32.25,32.28125,32.3125,32.34375,32.375,32.40625
0,1,1,1,80,0,1,-14.851402,-21.638525,-23.529105,-14.969324,...,-19.957216,-19.447732,-14.576655,-15.491358,-14.654316,-17.801658,-16.387551,-13.980891,-15.122689,-17.753853
1,2,1,1,160,0,2,-32.415520,-23.442737,-17.642596,-16.382163,...,-15.348137,-13.125596,-15.424900,-18.560355,-17.197693,-12.466164,-12.735882,-10.007690,-13.820587,-21.908636
2,3,1,1,240,0,3,-23.428437,-21.738735,-19.956032,-18.320372,...,-21.210045,-20.014100,-13.636612,-9.728106,-7.977970,-19.904029,-22.658427,-19.765777,-24.603827,-31.921519
3,4,1,1,320,0,4,-14.298403,-15.210905,-15.951684,-16.209202,...,-18.673789,-12.470911,-12.638057,-9.282286,-12.488977,-10.933173,-15.749634,-15.676254,-15.412170,-14.347517
4,5,1,1,400,0,5,-19.455200,-19.755724,-18.313774,-13.272877,...,-17.697814,-16.074418,-16.232349,-13.788018,-22.471867,-23.016590,-22.392253,-18.544159,-18.765642,-23.196090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,100,1,1680,0,21,-14.225847,-21.591727,-37.178597,-27.204617,...,-19.899728,-26.764563,-21.828736,-22.044932,-22.199481,-23.965621,-27.926072,-12.887907,-24.837898,-18.564334
2496,2496,100,1,1760,0,22,-6.786842,-19.817979,-13.008552,-28.766463,...,-19.512235,-25.466050,-24.668427,-19.157402,-26.706736,-27.000316,-32.832106,-21.871091,-21.461395,-29.887596
2497,2497,100,1,1840,0,23,-34.971051,-25.144396,-22.504613,-18.870299,...,-21.393051,-16.304893,-28.054869,-21.732752,-33.095179,-25.932350,-25.668614,-22.143077,-29.316773,-22.075520
2498,2498,100,1,1920,0,24,-16.954772,-32.034204,-20.303641,-24.144684,...,-24.076231,-23.104637,-24.846153,-22.963780,-23.709557,-26.729709,-30.922190,-24.463188,-33.253801,-27.997918


In [ ]:
def test_train_indexes_v2():
  train_idxs2 = []
  test_idxs2 = []
  for i in range(25):
      for j in range(25):
          test_idxs2.append(50*(i+1)-1-j)

  for i in range(25):
    for j in range(2):
      train_idxs2.append(50*i + j)
  return train_idxs2, test_idxs2

In [31]:
def get_idx_train_test_split(working_df, distances, trainingProp = 0.8):

  for d in distances:  
    distance_working_df = working_df.loc[working_df['Distance_km'] == d]

    rows_mixed=np.random.permutation(distance_working_df.shape[0])

    training_amt = math.ceil(distance_working_df.shape[0]*trainingProp)
    testing_amt = distance_working_df.shape[0] - training_amt

    rows_training = rows_mixed[:training_amt]
    rows_testing = rows_mixed[-testing_amt:]
    
  return rows_training, rows_testing

In [34]:
rows_training, rows_testing = get_idx_train_test_split(working_df, distances, trainingProp = 0.8)

# GD

## Gradient Descent Function

In [ ]:
def grad_b (M,m_og,m_tg,b,beta):
  return 2*beta*(M @ m_og + b - m_tg)

In [ ]:
def grad_M(M,cov_og,cov_tg,alpha):
    error = M @ cov_og @ np.transpose(M) - cov_tg  

    m1 = (2*error[0][0] * (2*cov_og[0][0]*M[0][0] + 2*cov_og[0][1]*M[0][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[1][0] + cov_og[0][1]*M[1][1]))
    
    m2 = (2*error[0][0] * (2*cov_og[1][1]*M[0][1] + 2*cov_og[0][1]*M[0][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[1][0] + cov_og[1][1]*M[1][1]))
    
    m3 = (2*error[1][1] * (2*cov_og[0][0]*M[1][0] + 2*cov_og[0][1]*M[1][1]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][0]*M[0][0] + cov_og[0][1]*M[0][1]))

    m4 = (2*error[1][1] * (2*cov_og[0][0]*M[1][1] + 2*cov_og[0][1]*M[1][0]) +
          2*(error[0][1] + error[1][0]) * (cov_og[0][1]*M[0][0] + cov_og[1][1]*M[0][1]))
    
    return alpha*np.array([[m1, m2], [m3, m4]])

In [ ]:
def gradient_descent(alpha,beta,m_tg,m_og,cov_tg,cov_og,nu,log):
  Ms = []
  bs = []
  M = np.random.rand(2,2)
  #M = np.array([[1, 0], [0, 1]])
  b = np.random.rand(2,1)
  #b = np.array([[0], [0]])
  #for i in range(100000):
  i = 0
  while True:
    # print('From:', M @ cov_source @ np.transpose(M))
    # print('To:', cov_target)
    L = loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b) 
    if  L < 1e-20:
        Ms.append(M)
        bs.append(b)
        break
    b = b - nu*grad_b(M,m_og,m_tg,b,beta)
    M = M - nu*grad_M(M,cov_og,cov_tg,alpha)
    i+= 1
    if (i>= 5000 and i <= 5025) or (i>= 5975 and i <= 6000):
      Ms.append(M)
      bs.append(b)
    if not i%5000 and log: print(L)
  if log: print("-"*25)
  return Ms,bs

## Compute parameters and mean/covariance

In [ ]:
def compute_mean_and_cov(data):
  aux_x = [] # Reales
  aux_y = [] # Imag
  for obs in data:
    aux_x.append(obs[0])
    aux_y.append(obs[1])
  return np.array([[np.mean(aux_x)],[np.mean(aux_y)]]), np.cov(aux_x,aux_y)


def total_loss(m_mod,m_tg,cov_mod,cov_tg):
  return (sum(sum(np.power(cov_mod-cov_tg,2))) + sum(np.power(m_mod - m_tg,2)))[0]


def loss(alpha,beta,m_tg,m_og,cov_tg,cov_og,M,b):
  first = alpha*sum(sum(np.power(M @ cov_og @ np.transpose(M) - cov_tg, 2)))
  second = beta*sum(np.power(M @ m_og + b - m_tg, 2))
  a = first+second
  return a

In [ ]:
def compute_parameters (const,method,source,target, params = None, log = False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  q_target = []

  for i in range(len(source)):

    q_source += [[x.real,x.imag] for x in source.values.tolist()[i] if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]
    q_target += [[x.real,x.imag] for x in target.values.tolist()[i]if x.real < limits[0][0] and x.real > limits[0][1] and x.imag < limits[1][0] and x.imag > limits[1][1]]

  unbiased = (len(q_source)/(len(q_source)-1) * 125/126)

  mean_source,cov_source = compute_mean_and_cov(q_source)
  mean_target, cov_target= compute_mean_and_cov(q_target)

  cov_source *= unbiased
  cov_target *= unbiased

  if method == "GD":
    if (params is None):
      alpha = 3/4
      beta = 1/4
    else:
      alpha = params["alpha"] if params["alpha"]>0 and params["alpha"]<=1 and params["alpha"]>params["beta"]  else 3/4
      beta = params["beta"] if params["beta"]>=0 and params["beta"]<1 and params["beta"]<params["alpha"]  else 1/4

    M, b = gradient_descent(alpha,beta,mean_target,mean_source,cov_target,cov_source, 0.5,log)
    return M,b

  elif method == "Z":
    return mean_source,cov_source,mean_target,cov_target

## Modify GD/Z

In [ ]:
def modify_const_GD(const,source,M,b, target = None, return_plots=False):

  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]
      
  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  for mult in mults:
    res = (M @ mult + b).tolist()

    # --------- Old line ---------
    #new_points.append([res[0][0],res[1][0]])
    nested_check = any(isinstance(i, list) for i in res[0])
    if nested_check:
      new_points.append([res[0][0][0],res[0][1][0]])
    else:
      new_points.append([res[0][0],res[1][0]])

  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      return new_points, q_target

  return new_points, indexes

In [ ]:
def modify_const_Z(const,source,mean_source,cov_source,mean_target,cov_target,target = None, return_plots=False):
  if const  == 1: limits = [[-2,-4],[4,2]]
  elif const == 7: limits = [[2,0],[2,0]]
  elif const == 10: limits = [[0,-2],[0,-2]]
  elif const == 15: limits = [[2,0],[-2,-4]]

  q_source = []
  indexes = []

  for i in range(len(source)):
    row_source = source.values.tolist()[i]
    for j in range(len(row_source)):
      point_source = row_source[j]
      if point_source.real < limits[0][0] and point_source.real > limits[0][1] and point_source.imag < limits[1][0] and point_source.imag > limits[1][1]:
        q_source += [[point_source.real,point_source.imag]]
        indexes.append([i,j])

  if target is not None:
    q_target = []

    for i in range(len(source)):
      row_target = target.values.tolist()[i]
      for j in range(len(row_source)):
        point_target = row_target[j]
        if point_target.real < limits[0][0] and point_target.real > limits[0][1] and point_target.imag < limits[1][0] and point_target.imag > limits[1][1]:
          q_target += [[point_target.real,point_target.imag]]

  mults = [np.array([[x[0]],[x[1]]]) for x in q_source]
  new_points = []

  w, v = np.linalg.eig(cov_source)
  S1 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)
  w, v = np.linalg.eig(cov_target)
  S2 = v @ np.diag(np.sqrt(w)) @ np.linalg.inv(v)

  for mult in mults:
    normalized = np.linalg.inv(S1) @ (mult - mean_source)
    denormalized = S2 @ normalized + mean_target

    new_points.append([denormalized[0][0], denormalized[1][0]])

  if target is not None:
    if return_plots:
      generate_comparison_plot(new_points,q_target)
      

  return new_points, indexes

In [ ]:
def L2dist(a,b):
    return math.sqrt(math.pow(a[0]-b[0],2)+math.pow(a[1]-b[1],2))

# Main

In [ ]:
def symbol2symbol_main(method, quadrants, input_data_path, distances, output_path, params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  
  if (params is None):
    output_features = "/{}_alpha_75_beta_25_features.csv".format(str(method))
  else:
    decimal_alpha = modf(params['alpha'])
    decimal_alpha  = round(decimal_alpha[0], 2)
    decimal_alpha = re.findall('..(.*)', str(decimal_alpha))[0]

    decimal_beta = modf(params['beta'])
    decimal_beta  = round(decimal_beta[0], 2)
    decimal_beta = re.findall('..(.*)', str(decimal_beta))[0]

    output_features = "/{}_alpha_{}_beta{}_features.csv".format(str(method), str(decimal_alpha), str(decimal_beta))

  # Get input data
  X, Y = create_df(input_data_path, distances, nsymbols, min_dist, max_dist)
  X = X.applymap(strToTuple)

 
  train_idxs2, test_idxs2 = test_train_indexes_v2()

  X_train2 = X.iloc[train_idxs2].reset_index(drop = "True")
  X_test2 = X.iloc[test_idxs2].reset_index(drop = "True") 

  data_new = {}

  for i in range(0,25):
    data_new[str(80*(i+1))] = {"Train":X_train2.iloc[i*2:(i*2+2)].reset_index(drop = "True"),"Test":X_test2.iloc[i*25:(i*25+25)].reset_index(drop = "True")}  

  first = True
  modify_all_data_df = pd.DataFrame()
  for i in tqdm(range(25)):
      print('\n')
      print("i = ",i)
      modify_data_df = pd.DataFrame()
      for j in range(i+1,25):
          print(" j = ",j)
          source = data_new[str(distances[i]*80)]['Train']
          target = data_new[str(distances[j]*80)]['Train']
          source_test = data_new[str(distances[i]*80)]['Test']

          source_test2 = source_test.copy()

          for quadrant in quadrants:
              print("   quadrant = ", quadrant)
              if method == "GD":
                M, b = compute_parameters(quadrant, 'GD', source, target, params=params)
                new_points, indexes = modify_const_GD(quadrant, source_test, M, b)
              elif method == "Z":
                mean_source,cov_source,mean_target,cov_target = compute_parameters(quadrant, 'Z', source, target, params=None)
                new_points, indexes = modify_const_Z(quadrant,source_test,mean_source,cov_source,mean_target,cov_target)

              for k in range(len(indexes)):
                  source_test2.iloc[indexes[k][0],indexes[k][1]] = complex(new_points[k][0], new_points[k][1])

          F=[]
          for k in range(source_test2.shape[0]):
              data2=list(source_test2.iloc[k,:])
              data2=[[float(d.real), float(d.imag)] for d in data2]
              gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data2)
              mus=gmm.means_
              sigmas=gmm.covariances_

              features=[distances[i]*80, distances[j]*80]

              for z in selCP_pos:
                  mindist=None
                  k_inc=None
                  for w in range(16):
                      d=L2dist(mus[w],z)
                      if mindist is None or mindist>d:
                          mindist=d
                          k_inc=w

                  covmat=np.concatenate(list(sigmas[k_inc])).ravel().tolist()
                  features = [*features, *mus[k_inc], *covmat]
              F.append(features)
          
          header=['original_dist', 'target_dist']
          
          for j in selCP:
              header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ri_'+str(j),'sigma_ir_'+str(j),'sigma_ii_'+str(j)]]
   
          with open(output_path + output_features, 'a', encoding='UTF8', newline='') as f:
              writer = csv.writer(f)
              # write the header
              if first:
                writer.writerow(header)
                first = False
              # write multiple rows
              writer.writerows(F)   

# Main v2

In [ ]:
# input_data_path = path + "/Spectrum/CNN/new_working_df.csv"
def symbol2symbol_main_v2(method, quadrants, input_data_path, distances, output_path, params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000):
  
  if (params is None):
    output_features = "/{}_alpha_75_beta_25_features.csv".format(str(method))
  else:
    decimal_alpha = modf(params['alpha'])
    decimal_alpha  = round(decimal_alpha[0], 2)
    decimal_alpha = re.findall('..(.*)', str(decimal_alpha))[0]

    decimal_beta = modf(params['beta'])
    decimal_beta  = round(decimal_beta[0], 2)
    decimal_beta = re.findall('..(.*)', str(decimal_beta))[0]

    output_features = "/{}_alpha_{}_beta{}_features.csv".format(str(method), str(decimal_alpha), str(decimal_beta))
  
  # Get input data
  working_df = pd.read_csv(input_data_path)
  working_df['Distance_km'] = working_df['Distance_km'].astype(int)

  #Get train/test split
  train_idxs2, test_idxs2 = get_idx_train_test_split(working_df, distances, trainingProp = 0.8)
  X_train2 = X.iloc[train_idxs2].reset_index(drop = "True")
  X_test2 = X.iloc[test_idxs2].reset_index(drop = "True")

  # Get labels and data
  distances = [x*80 for x in range(1, 26)]
  X, Y = create_df_v2(working_df, distances, nsymbols = 2048, span_length=80,  min_dist = 0, max_dist = 3000)
 

  data_new = {}

  for i in range(0,25):
    data_new[str(80*(i+1))] = {"Train":X_train2.iloc[i*2:(i*2+2)].reset_index(drop = "True"),"Test":X_test2.iloc[i*25:(i*25+25)].reset_index(drop = "True")}  

  first = True
  modify_all_data_df = pd.DataFrame()
  for i in tqdm(range(25)):
      print('\n')
      print("i = ",i)
      modify_data_df = pd.DataFrame()
      for j in range(i+1,25):
          print(" j = ",j)
          source = data_new[str(distances[i]*80)]['Train']
          target = data_new[str(distances[j]*80)]['Train']
          source_test = data_new[str(distances[i]*80)]['Test']

          source_test2 = source_test.copy()

          for quadrant in quadrants:
              print("   quadrant = ", quadrant)
              if method == "GD":
                M, b = compute_parameters(quadrant, 'GD', source, target, params=params)
                new_points, indexes = modify_const_GD(quadrant, source_test, M, b)
              elif method == "Z":
                mean_source,cov_source,mean_target,cov_target = compute_parameters(quadrant, 'Z', source, target, params=None)
                new_points, indexes = modify_const_Z(quadrant,source_test,mean_source,cov_source,mean_target,cov_target)

              for k in range(len(indexes)):
                  source_test2.iloc[indexes[k][0],indexes[k][1]] = complex(new_points[k][0], new_points[k][1])

          F=[]
          for k in range(source_test2.shape[0]):
              data2=list(source_test2.iloc[k,:])
              data2=[[float(d.real), float(d.imag)] for d in data2]
              gmm = GaussianMixture(n_components=16, random_state=0, means_init=my_centers).fit(data2)
              mus=gmm.means_
              sigmas=gmm.covariances_

              features=[distances[i]*80, distances[j]*80]

              for z in selCP_pos:
                  mindist=None
                  k_inc=None
                  for w in range(16):
                      d=L2dist(mus[w],z)
                      if mindist is None or mindist>d:
                          mindist=d
                          k_inc=w

                  covmat=np.concatenate(list(sigmas[k_inc])).ravel().tolist()
                  features = [*features, *mus[k_inc], *covmat]
              F.append(features)
          
          header=['original_dist', 'target_dist']
          
          for j in selCP:
              header=[*header,*['mu_r_'+str(j),'mu_i_'+str(j),'sigma_rr_'+str(j),'sigma_ri_'+str(j),'sigma_ir_'+str(j),'sigma_ii_'+str(j)]]
   
          with open(output_path + output_features, 'a', encoding='UTF8', newline='') as f:
              writer = csv.writer(f)
              # write the header
              if first:
                writer.writerow(header)
                first = False
              # write multiple rows
              writer.writerows(F)   

# Run

In [ ]:
# PLEASE SELCET THE METHOD AND QUADRANTS THAT ARE DESIRED TO BE TESTED
method = "GD" # or method = "Z"
quadrants = [1, 7, 10, 15]

# PLEASE review that the path below is correct
input_data_path=path+"/rawData"
print('filepath selected: {}'.format(input_data_path))

#Output mod files
output_path = path + '/modifiedData/'+ method

# Params for gradient descent alpha and beta
a = 0.7
b = 0.2
params = {"alpha": float(a), "beta": float(b)}

In [ ]:
symbol2symbol_main(method, quadrants, input_data_path, distances, output_path, mod_csv = True, mod_original = False,  params = None, nsymbols = 2048, min_dist = 0, max_dist = 3000)